In [1]:
# Paso 0: Instalación (si estás en un entorno local)
# !pip install torch torchvision

### 📌 Paso 1: Importar PyTorch
Importamos los módulos principales de PyTorch. `torch` para tensores y cálculo, `nn` para redes neuronales, `optim` para los optimizadores y `datasets`/`transforms` para manejar datos de entrenamiento.

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

### 🔍 Paso 2: Tensores — La base de PyTorch
Los tensores son como arrays de NumPy, pero pueden ejecutarse en GPU. Aquí mostramos cómo crearlos, realizar operaciones, y cambiar su tipo.

In [3]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
print("Tensor a:", a)
print("Tensor b:", b)
print("Suma:", a + torch.tensor([3, 2, 1]))
print("Multiplicación:", b * 2)
print("Convertir a float:", a.float())

Tensor a: tensor([1, 2, 3])
Tensor b: tensor([[1., 2.],
        [3., 4.]])
Suma: tensor([4, 4, 4])
Multiplicación: tensor([[2., 4.],
        [6., 8.]])
Convertir a float: tensor([1., 2., 3.])


### 🧮 Paso 3: Requiere gradiente para entrenamiento
Si un tensor tiene `requires_grad=True`, PyTorch calculará automáticamente los gradientes necesarios para entrenamiento de modelos.

In [4]:
x = torch.tensor(2.0, requires_grad=True)
y = x**2 + 3*x + 1
y.backward()
print("Gradiente dy/dx:", x.grad)

Gradiente dy/dx: tensor(7.)


### 📚 Paso 4: Dataset y DataLoader
Se usa `datasets.MNIST` para obtener datos de imágenes escritas a mano, y `DataLoader` para cargar los datos en lotes.

In [5]:
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)
print("Shape del lote:", example_data.shape)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:12<00:00, 813kB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 356kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 2.51MB/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 4.52MB/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Shape del lote: torch.Size([64, 1, 28, 28])


### 🧠 Paso 5: Definir un modelo simple (MLP)
Creamos una red neuronal simple con una capa oculta. Usamos `nn.Module` como base.

In [6]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Aplanar
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = NeuralNet()

### ⚙️ Paso 6: Función de pérdida y optimizador
La función de pérdida mide cuán mal está el modelo. El optimizador actualiza los pesos.

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 🔁 Paso 7: Entrenar el modelo
Por cada época, el modelo hace predicciones, calcula el error, propaga el gradiente hacia atrás y actualiza los pesos.

In [8]:
num_epochs = 1
for epoch in range(num_epochs):
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/1], Loss: 0.3153


### 🧪 Paso 8: Evaluar el modelo
Se evalúa el modelo sin calcular gradientes, para medir su precisión.

In [9]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in train_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Precisión del modelo: {100 * correct / total:.2f}%')

Precisión del modelo: 95.01%


### 💾 Paso 9: Guardar y cargar el modelo
Para usar el modelo en el futuro, lo guardamos y mostramos cómo cargarlo.

In [10]:
torch.save(model.state_dict(), "modelo_mnist.pth")
modelo_cargado = NeuralNet()
modelo_cargado.load_state_dict(torch.load("modelo_mnist.pth"))
modelo_cargado.eval()

C:\Users\PC MASTER\AppData\Local\Temp\ipykernel_25428\249456814.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  modelo_cargado.load_state_dict(torch.load("modelo_mnist.p

NeuralNet(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)